In [6]:
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io
import gc
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
import tensorflow as tf
import seaborn as sns
from scipy.signal import butter, lfilter
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Flatten, MaxPooling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.activations import relu, tanh, linear
from tensorflow.keras.utils import Progbar
from joblib import Parallel, delayed #Paralleize calculation
import psycopg2
from sqlalchemy import create_engine, Column, Integer, ARRAY, MetaData, Table, Text
from sqlalchemy.dialects.postgresql import ARRAY as PG_ARRAY
from psycopg2.extensions import register_adapter, AsIs
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.backend import clear_session

# Set random seed for reproducability
tf.keras.utils.set_random_seed(42)
# check if tensorflow uses GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [12]:
# Custom adapter function for postgre
def adapt_numpy_ndarray(numpy_array):
    return AsIs(list(numpy_array))
# Register the postgre-adapter
register_adapter(np.ndarray, adapt_numpy_ndarray)

# Database connection parameters and alchemy engine
dbname = 'bathunting'
user = 'python'
password = 'python_password'
host = 'localhost'
port = '5432' 
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')

# Functions to get datatf.compat.v1.reset_default_graph()  # Reset the TensorFlow graph

def get_target_data(target, limit=0, no_target=False):
    lmt = "" if limit<=0 else f"LIMIT {limit}"
    #query = ""
    if no_target:
        query = f"SELECT new_arr FROM batcall where 10 < ANY(new_arr) and target = {target} {lmt}"
    else:
        query = f"SELECT target, new_arr FROM batcall where 10 < ANY(new_arr) and target = {target} {lmt}"
    df = pd.read_sql_query(query, engine)
    if no_target:
        df = pd.DataFrame(df['new_arr'].tolist())
    return df

# messy, i know
def get_targets_to_data(limit=0):
    lmt = "" if limit<=0 else f"LIMIT {limit}"
    query = f"SELECT target FROM batcall where 10 < ANY(new_arr) {lmt}"
    df = pd.read_sql_query(query, engine)
    return df

def get_all_data(targets=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18], limit=0, no_target=False):
    all_df = Parallel(n_jobs=-3, prefer="threads")(delayed(get_target_data)(target, limit, no_target) for target in targets)
    return all_df

def get_data(targets=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18], limit=0, no_target=False):
    all_df = Parallel(n_jobs=-3, prefer="threads")(delayed(get_target_data)(target, limit, no_target) for target in targets)
    df = pd.concat(all_df)
    return df

# try to visualize only the maximum values per species
def spectrogram_range(target):
    df = get_target_data(target, limit=0, no_target=True)
    max_vals = df.max()
    min_vals = df.min()
    
    abs_pos = max_vals.abs()
    abs_neg = min_vals.abs()
    mask = (abs_pos > abs_neg).astype(bool)
    return min_vals.where(mask, max_vals), max_vals.where(mask, min_vals)

def get_targets():
    #conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
    #cursor = conn.cursor()
    query = f"SELECT target, bat FROM batcall group by target, bat order by target"
    df = pd.read_sql_query(query, engine)
    #conn.close()
    return df

def get_shape(nested_list):
    try:
        # Initialize shape list
        shape = []

        # Iterate to calculate the shape
        while isinstance(nested_list, list) or isinstance(nested_list, np.ndarray):
            shape.append(len(nested_list))
            nested_list = nested_list[0]

        return tuple(shape)
    except (TypeError, IndexError) as e:
        # In case the nested lists are not uniformly sized
        return f"Irregular shape - nested lists are not of equal size. \n ERROR: {e}"
    
# Get data to work with
def get_features_and_targets(limit=100, scaler=StandardScaler(), categorical=True):
    data = get_data(limit=limit)

    df = pd.DataFrame(data["new_arr"].tolist())
    if scaler != None:
        df = scaler.fit_transform(df)

    labels = pd.DataFrame(data["target"])
    if categorical:
        labels = to_categorical(labels, num_classes=19)
    return df, labels

def vogl_conversion(df):
    data_reshaped = []
    for _,data in df.iterrows():
        # Normalize
        data -= np.mean(data)
        data /= np.std(data)
        # Realy no idea just assuming prof did it right
        # Calculate spectrogram with FFT
        stft = np.abs(librosa.stft(np.array(data), n_fft=512, hop_length=32))
        stft = 10 * np.log10(stft)
        stft = np.nan_to_num(stft)
        # Scale between [0,1] and reduce shape if needed
        stft = (stft - np.min(stft)) / (np.max(stft) - np.min(stft))
        stft = np.reshape(stft, (257, 138, 1))
        stft = stft[:256, -128: , :]
        data_reshaped.append(stft)
    return np.array(data_reshaped)

def confusion_matrix(model, X_test, y_test):
    # Confusion Matrix
    predictions = model.predict(X_test)
    predicted_labels = np.argmax(predictions, axis=1)
    true_labels = np.argmax(y_test, axis=1)  # y_test are the true labels (one-hot encoded)
    conf_matrix = confusion_matrix(true_labels, predicted_labels)

    # Plot the confusion matrix using Seaborn
    sns.heatmap(conf_matrix, annot=True, fmt='g')
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.show()

def visualize_history(history, title_appendix=''):
    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'Model accuracy {title_appendix}')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.savefig(f'History/accuracy_{title_appendix}.png')
    plt.close()
    
    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'Model loss {title_appendix}')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.savefig(f'History/loss_{title_appendix}.png')
    plt.close()

This should be all about Convolutional Neuronal Networks and lowering the loss

In [7]:
df, labels = get_features_and_targets(limit=500, scaler=None)
df_reshaped = vogl_conversion(df)
X_train, X_test, y_train, y_test = train_test_split(df_reshaped, labels, test_size=0.2, random_state=42)
X_train = X_train[:-3]
y_train = y_train[:-3]
print(get_shape(X_train))
print(get_shape(y_train))
# Determine the number of input features
input_dim = get_shape(X_train)

/tmp/ipykernel_16710/3122854350.py:101: RuntimeWarning: divide by zero encountered in log10
  stft = 10 * np.log10(stft)


(7250, 256, 128, 1)
(7250, 19)


In [13]:
# Density 50 to 5000 -> does not realy matter after 100 and not matter anymore after 500
for i in range(50, 5001, 50):
    clear_session() # clear keras session
    tf.compat.v1.reset_default_graph()  # Reset the TensorFlow graph
    #tf.config.gpu.set_per_process_memory_fraction(0.5)
    # Build the model
    model = Sequential()
    model.add(Conv2D(16, (3,3), activation='relu', padding='same', input_shape=(256,128,1)))
    model.add(MaxPooling2D(2, 2))
    model.add(Flatten())
    model.add(Dense(i, activation='relu'))
    model.add(Dense(19, activation='softmax'))

    # Compile the model
    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(learning_rate=0.001),
                  metrics=['accuracy'])

    # Fit the model
    history = model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=0, validation_split=0.2)

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print('Density: %.i' % (i))
    print(f"Final Training Loss: {history.history['loss'][-1]}")
    print(f"Final Training Accuracy: {history.history['accuracy'][-1]}")
    print(f"Final Validation Loss: {history.history['val_loss'][-1]}")
    print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]}")
    print('Test Accuracy: %.2f %%' % (accuracy*100))
    print('Test Loss: %.2f' % (loss))
    visualize_history(history, f'dens={i}_finacc={history.history["val_accuracy"][-1]:.2f}')
    
    # Free Memory for following models
    del model
    gc.collect()
    tf.compat.v1.reset_default_graph()  # Reset the TensorFlow graph

57/57 [==============================] - 0s 6ms/step - loss: 2.9404 - accuracy: 0.0485
Density: 50
Final Training Loss: 2.9372239112854004
Final Training Accuracy: 0.05155172571539879
Final Validation Loss: 2.9394729137420654
Final Validation Accuracy: 0.05517241358757019
Test Accuracy: 4.85 %
Test Loss: 2.94
57/57 [==============================] - 0s 5ms/step - loss: 3.0835 - accuracy: 0.6985
Density: 100
Final Training Loss: 1.479933757764229e-06
Final Training Accuracy: 1.0
Final Validation Loss: 2.7108476161956787
Final Validation Accuracy: 0.7103448510169983
Test Accuracy: 69.85 %
Test Loss: 3.08
57/57 [==============================] - 0s 5ms/step - loss: 1.7402 - accuracy: 0.7800
Density: 150
Final Training Loss: 6.372534357979021e-07
Final Training Accuracy: 1.0
Final Validation Loss: 1.7338581085205078
Final Validation Accuracy: 0.7731034755706787
Test Accuracy: 78.00 %
Test Loss: 1.74
57/57 [==============================] - 1s 7ms/step - loss: 1.7007 - accuracy: 0.7778
Dens

KeyboardInterrupt: 

In [14]:
# Clear grafic memory
clear_session() # clear keras session
tf.compat.v1.reset_default_graph()  # Reset the TensorFlow graph
# Free Memory for following models
del model
gc.collect()
tf.compat.v1.reset_default_graph()  # Reset the TensorFlow graph